In [1]:
import glob #to read the files
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.interpolate import interp1d
import os.path 
from tabulate import tabulate #to export in table format

In [2]:
X_ = np.linspace(-10, 50, 60 + 1)
wave = np.linspace(3300, 8500, 520 + 1)
list_data = [[]for y in range(0,521)]
save_path = '/home/joao/Documentos/PCA'

In [3]:
file = open("names.txt")
lines = file.readlines()
file.close()

In [4]:
training_set = []

In [5]:
for line in lines:
    if line.startswith('#'): continue
    co=line.rstrip().replace('INDEF','Nan').split()

    training_set.append(co[0])

In [6]:
score = []

In [ ]:
for j in range(0,len(training_set)):

    sn_names=glob.glob("/home/joao/Documentos/templates/data_ext/"+training_set[j]+"*"+".dat")
    
    for p in range(0,len(sn_names)):

        sn_names[p] = sn_names[p][41:]
    
    
    print(training_set[j])
    
    print(sn_names)
    
    if len(sn_names) > 3:
        
        
        
        for i in range(0,len(sn_names)):

            y = []
            z = []

            file = open(sn_names[i])
            lines = file.readlines()
            file.close()

            for line in lines:
                if line.startswith('#'): continue
                co=line.rstrip().replace('INDEF','Nan').split()

                y.append(co[0])
                z.append(co[1])


            #y = np.array(y, dtype=float)
            z = np.array(z, dtype=float)

            timetemp = lines[0]
            timetemp = timetemp[:-1]
            timetemp = float(timetemp[1:])

            f = max(z)

            for jj in range(0,len(z)):

                list_data[jj].append([timetemp,z[jj]])

            #print(i)

        Export = []

        for jj in range(0,len(list_data)):

            yy = []
            zz = []

            for k in range(0,len(list_data[jj])):

                yy.append([list_data[jj][k][0]])
                zz.append(list_data[jj][k][1])

            ff = max(zz)

            zz = zz/ff

            kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(0.01, 10)) + WhiteKernel(noise_level=1e-1, noise_level_bounds=(1e-10, 1e+1))

            gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(yy, zz)

            val_score = gp.score(yy, zz)
            score.append(val_score)

            #y_mean = gp.predict(X_[:, np.newaxis])

            #for l in range(0,len(y_mean)):

                #Export.append(y_mean[l]*ff)
        
        
        '''    
        table = []
        for l in range(0,len(Export)):
            table.append((Export[l],training_set[j]))


        name_of_file = training_set[j]

        completeName = os.path.join(save_path, name_of_file+".dat")

        f = open(completeName, 'w')  
        f.write(tabulate(table, tablefmt="plain"))
        f.close()
        '''

SN2012fr
[]
SN2005cf
['SN2005cf3.dat', 'SN2005cf27.dat', 'SN2005cf13.dat', 'SN2005cf0.dat', 'SN2005cf22.dat', 'SN2005cf5.dat', 'SN2005cf4.dat', 'SN2005cf28.dat', 'SN2005cf16.dat', 'SN2005cf33.dat', 'SN2005cf35.dat', 'SN2005cf17.dat', 'SN2005cf12.dat', 'SN2005cf7.dat', 'SN2005cf21.dat', 'SN2005cf11.dat', 'SN2005cf34.dat', 'SN2005cf25.dat', 'SN2005cf6.dat', 'SN2005cf9.dat', 'SN2005cf15.dat', 'SN2005cf14.dat', 'SN2005cf18.dat', 'SN2005cf32.dat', 'SN2005cf10.dat', 'SN2005cf30.dat', 'SN2005cf26.dat']
SN2001V
['SN2001V17.dat', 'SN2001V5.dat', 'SN2001V12.dat', 'SN2001V34.dat', 'SN2001V7.dat', 'SN2001V33.dat', 'SN2001V29.dat', 'SN2001V18.dat', 'SN2001V37.dat', 'SN2001V25.dat', 'SN2001V10.dat', 'SN2001V26.dat', 'SN2001V31.dat', 'SN2001V11.dat', 'SN2001V9.dat', 'SN2001V1.dat', 'SN2001V21.dat', 'SN2001V24.dat', 'SN2001V36.dat', 'SN2001V14.dat', 'SN2001V27.dat', 'SN2001V30.dat', 'SN2001V4.dat', 'SN2001V16.dat', 'SN2001V32.dat', 'SN2001V19.dat', 'SN2001V3.dat', 'SN2001V6.dat', 'SN2001V15.dat', 'SN2

In [ ]:
%matplotlib inline
plt.hist(score, bins=30)
plt.show()